# Regression with ARIMA Errors.

This model is a simple Regression model with ARIMA errors.  The regression model consists of a single independent variables 'new years day'.  This is a categorical variable to model the special event on 1st Jan every year.

ARIMA order is chosen by `pmdarima.auto_arima`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

#auto_arima
from pmdarima import auto_arima

from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels as sm
sm.__version__

'0.11.0'

In [3]:
import pmdarima as pm
pm.__version__

'1.5.2'

In [4]:
from amb_forecast.feature_engineering import (featurize_time_series,
                                              regular_busy_calender_days)

# Data Input

In [5]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'reg-arima'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [6]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [7]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [8]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [9]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [10]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#split into train and val AFTER creating new years day.


In [11]:
train.shape

(1828, 9)

In [12]:
test.shape

(365, 9)

# New years day

Generate a new binary categorical feature representing new years day.

In [13]:
#exclude interaction as point forecasts are less accurate.
_, _, exog = featurize_time_series(train[REGION], max_lags=1, 
                                   include_interactions=False)

In [14]:
exog.head()

,top
actual_dt,
2013-12-30,0
2013-12-31,0
2014-01-01,1
2014-01-02,0
2014-01-03,0


In [15]:
#combine exog into train array for split
train['new_year'] = exog

In [16]:
train.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire,new_year
actual_dt,,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0,0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0,0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0,1
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0,0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0,0


# Train validation split

In [17]:
#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)

In [18]:
train.shape

(1279, 10)

In [19]:
val.shape

(549, 10)

# Auto ARIMA model selection

Uses Auto ARIMA function to select model by AIC


In [20]:
##only looking at Trust level.
#auto_results = auto_arima(train[REGION], 
#                          exogenous=train['new_year'].to_numpy().reshape(-1, 1), 
#                          seasonal=True, 
#                          m=7, 
#                          n_job=-1, 
#                          suppress_warnings=False) 

In [21]:
#auto_results.summary()

## Test Out of Sample Prediction

In [22]:
print("pmdarima version: %s" % pm.__version__)
#print(auto_results.order)

pmdarima version: 1.5.2


In [23]:
#override if desired to test different model.
#order = auto_results.order
#seasonal_order = auto_results.seasonal_order
order = (1, 1, 2)
seasonal_order = (1, 0, 1, 7)


In [24]:
model = pm.ARIMA(order=order, seasonal_order=seasonal_order)

In [25]:
model.fit(y=train[REGION], 
          exogenous=train['new_year'].to_numpy().reshape(-1, 1))

ARIMA(maxiter=50, method='lbfgs', order=(1, 1, 2), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(1, 0, 1, 7),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=True)

In [26]:
#test prediction
model.predict(n_periods=5, 
              exogenous=val['new_year'].iloc[:5].to_numpy().reshape(-1, 1))

array([2262.30505229, 2244.11424738, 2152.94249696, 2082.20896898,
       2078.16437783])

# Wrapper classes for statsmodels ARIMA

Adapter/wrapper class to enable usage within standard cross validation used across all methods

In [27]:
class ARIMAWrapper:
    '''
    Facade for statsmodels ARIMA models
    '''
    def __init__(self, order, seasonal_order, training_index, holidays=None):
        self._order = order
        self._seasonal_order = seasonal_order
        self._training_index = training_index
        self._holidays = holidays

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues
    
    def _encode_holidays(self, holidays, idx):
        dummy = idx.isin(holidays).astype(int)
        dummy = pd.DataFrame(dummy)
        dummy.columns = ['holiday']
        dummy.index = idx
        return dummy

    def fit(self, y_train):
        
        #extend training index
        if len(y_train) > len(self._training_index):

            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
            
        holiday_train = None
        if not self._holidays is None:
            holiday_train = self._encode_holidays(self._holidays, 
                                                  self._training_index)
    
        
        self._model = ARIMA(endog=y_train,
                            exog=holiday_train,
                            order=self._order, 
                            seasonal_order=self._seasonal_order)#,
                            #enforce_stationarity=False)
        
        self._fitted = self._model.fit()
        self._t = len(train)
        
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        
        #+1 to date range then trim off the first value

        f_idx = pd.date_range(start=self._training_index[-1], 
                              periods=horizon+1,
                              freq=self._training_index.freq)[1:]
        
        #encode holidays if included.
        exog_holiday = None
        if not self._holidays is None:
            exog_holiday = self._encode_holidays(self._holidays, f_idx)
        
    
        forecast = self._fitted.get_forecast(horizon, exog=exog_holiday)
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi
            
        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)  

## Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [28]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days
    
    alpha - float, optional (default=0.2)
        1 - alpha prediction interval specification

    step -- int, optional (default=1)
            step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array, np.array, np.array
        - cv_preds, cv_test, cv_intervals
    '''
    
    #point forecasts
    cv_preds = [] 
    #ground truth observations
    cv_actuals = [] 
    #prediction intervals
    cv_pis = []
    
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
                
        train_cv = np.concatenate([train, val[:i]], axis=0)
        model.fit(train_cv)
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=len(val[i:i+max(horizons)]), 
                                   return_conf_int=True,
                                   alpha=alpha)        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        #sub horizon calculations
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [29]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [30]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

# Run cross validation

This is run twices once each for 80 and 95% prediction intervals.

In [31]:
#reminder of ARIMA order
print(order)
print(seasonal_order)

(1, 1, 2)
(1, 0, 1, 7)


## Holidays included

### New years day

In [32]:
exog = regular_busy_calender_days(train[REGION], quantile=0.99)

In [33]:
new_year = pd.DataFrame({
                         'holiday': 'new_year',
                         'ds': pd.date_range(start=exog[0], 
                                             periods=20, 
                                             freq='YS')
                        })

### Create model and run

In [34]:
model = ARIMAWrapper(order=order, seasonal_order=seasonal_order, 
                     training_index=train.index, 
                     holidays=new_year['ds'].tolist())

horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
results = time_series_cv(model, train[REGION], val[REGION], 
                         horizons, alpha=0.2, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



## symmetric MAPE results

In [35]:
cv_preds, cv_test, cv_intervals = results

In [36]:
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,2.817042,3.164754,3.377149,3.531989,3.695879,3.849643,3.982054,4.088984,4.203878,4.293219,4.368358,4.448976,5.575053
std,0.958813,0.962132,0.947362,0.880134,0.944026,0.986216,1.015735,0.980011,1.048592,1.055287,1.056161,1.146978,2.037696
min,1.463270,1.849534,2.359003,2.465616,2.664897,2.656501,2.526379,2.645832,2.756132,2.905157,2.979387,2.990024,3.743676
25%,2.189718,2.645513,2.755919,2.970553,3.107871,3.210227,3.303151,3.490358,3.625386,3.627445,3.580980,3.595365,4.353181
50%,2.570574,2.993214,3.235118,3.387836,3.397949,3.578998,3.811569,3.862348,3.887043,4.170335,4.331648,4.318468,5.050290
75%,3.360787,3.471712,3.643612,3.817096,4.157899,4.397981,4.449756,4.542215,4.592055,4.776573,4.902100,5.026401,5.891917
max,5.538577,6.005663,6.657392,6.526569,7.041053,7.186746,7.257271,7.035648,7.449388,7.213427,7.029521,7.463354,13.100792


In [37]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima_smape.csv


## RMSE results

In [38]:
#CV point predictions rmse - no interactions
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,74.379376,84.992675,91.967937,96.626805,101.417551,105.729130,109.535798,112.767167,116.084131,118.890073,121.294654,124.008786,153.554509
std,25.067928,26.956111,26.990742,25.824350,26.697122,27.139625,27.640610,26.882039,28.468424,28.510738,28.437835,30.650932,48.406822
min,38.074127,51.713581,63.267767,68.387649,71.872953,74.509067,72.564137,74.011975,75.259112,78.740762,82.169201,81.988381,108.684579
25%,58.603831,68.967503,74.296010,81.693992,85.046496,87.203760,89.340316,93.995922,99.465171,99.541410,101.672388,103.535723,127.098163
50%,70.301998,76.765234,85.178126,89.493967,89.627893,97.350091,105.518806,106.712349,110.973125,117.291884,121.885571,122.197493,139.470141
75%,87.445541,92.344350,91.310916,99.510305,108.258282,121.945788,129.215219,128.740601,127.810914,132.583350,135.423936,139.876795,159.012268
max,155.583825,158.381710,174.160152,168.925672,180.084651,182.188267,183.652824,179.035187,189.352490,189.276479,182.503377,194.250946,335.478567


In [39]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima_rmse.csv


## MASE results

In [40]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train['Trust'])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.758820,0.854923,0.914108,0.957469,1.003028,1.045750,1.082595,1.112239,1.143974,1.168610,1.189556,1.212100,1.520025
std,0.280369,0.292226,0.291738,0.277567,0.294696,0.305287,0.311005,0.299490,0.314455,0.312985,0.309346,0.331076,0.594339
min,0.391092,0.472348,0.594703,0.651643,0.678757,0.683521,0.647589,0.677585,0.706520,0.748448,0.774617,0.778185,1.017615
25%,0.570231,0.666615,0.716045,0.785877,0.820397,0.832725,0.879201,0.926893,0.959506,0.966033,0.991836,0.989936,1.175340
50%,0.699802,0.808350,0.840069,0.892695,0.900337,0.953938,1.053282,1.073590,1.080745,1.165237,1.177223,1.132682,1.364792
75%,0.874319,0.915400,0.962886,1.018530,1.124226,1.208289,1.239008,1.280964,1.266780,1.304038,1.352616,1.343522,1.572372
max,1.631128,1.726549,1.898614,1.862760,2.003457,2.044160,2.062336,1.999032,2.112901,2.047065,1.998356,2.119338,3.758698


In [41]:
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima_mase.csv


## 80% Coverage

In [42]:
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.804233,0.788360,0.765432,0.744709,0.722751,0.709877,0.696145,0.691138,0.686067,0.679365,0.677730,0.673721,0.728463
std,0.177848,0.162739,0.148206,0.137860,0.141047,0.144230,0.144677,0.140242,0.146364,0.149690,0.147286,0.156104,0.155323
min,0.428571,0.285714,0.285714,0.321429,0.257143,0.238095,0.244898,0.267857,0.238095,0.271429,0.311688,0.285714,0.205479
25%,0.714286,0.714286,0.738095,0.678571,0.657143,0.642857,0.642857,0.642857,0.626984,0.614286,0.616883,0.577381,0.639726
50%,0.857143,0.785714,0.809524,0.785714,0.771429,0.714286,0.714286,0.696429,0.714286,0.685714,0.701299,0.690476,0.750685
75%,1.000000,0.892857,0.857143,0.839286,0.828571,0.833333,0.806122,0.785714,0.793651,0.807143,0.805195,0.797619,0.842466
max,1.000000,1.000000,0.904762,0.892857,0.914286,0.880952,0.877551,0.875000,0.888889,0.871429,0.844156,0.857143,0.915068


In [43]:
#write 80% coverage to file
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima_coverage_80.csv


# Repeat for 95% PIs.

In [65]:
#95% PIs
model = ARIMAWrapper(order=order, seasonal_order=seasonal_order, 
                     training_index=train.index, 
                     holidays=new_year['ds'].tolist())

horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
results = time_series_cv(model, train[REGION], val[REGION], 
                         horizons, alpha=0.05, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



## 95% coverage results

In [66]:
cv_preds, cv_test, cv_intervals = results

In [67]:
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.962963,0.939153,0.929453,0.929894,0.922751,0.914462,0.908541,0.902778,0.893004,0.887831,0.882155,0.878307,0.909285
std,0.075099,0.111546,0.113896,0.100275,0.104172,0.103206,0.092553,0.083366,0.093346,0.090064,0.091685,0.100492,0.062361
min,0.714286,0.571429,0.523810,0.571429,0.514286,0.500000,0.551020,0.607143,0.571429,0.614286,0.649351,0.642857,0.734247
25%,1.000000,0.928571,0.904762,0.928571,0.900000,0.880952,0.857143,0.866071,0.857143,0.857143,0.857143,0.845238,0.863014
50%,1.000000,1.000000,0.952381,0.964286,0.942857,0.928571,0.938776,0.910714,0.904762,0.900000,0.896104,0.904762,0.923288
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.976190,0.979592,0.964286,0.968254,0.950000,0.948052,0.952381,0.969863
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.978082


In [68]:
#write 95% coverage to file
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima_coverage_95.csv


# End